<a href="https://colab.research.google.com/github/FernandoBRdgz/inteligencia_artificial/blob/main/grandes_modelos_de_lenguaje/llms_con_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a LangChain

LangChain es un marco para desarrollar aplicaciones impulsadas por modelos de lenguaje.

**Objetivos** Revisión de los siguientes conceptos:

- LLMs (Grandes modelos de lenguaje)
- Prompt Templates (Plantillas)
- Chains (Cadenas)
- Agents and Tools (Agentes y herramientas)
- Memory (Memoria)
- Document Loaders (Cargadores de documentos)
- Indexes (Índices)

### Instalación

In [1]:
!pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.3/518.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.9
    Uninstalling SQLAlchemy-2.0.9:
      Successfully uninstalled SQLAlchemy-2.0.9


## 1. LLMs

Una interfaz genérica para todos los LLM. Ver todos los proveedores de LLM: https://python.langchain.com/en/latest/modules/models/llms/integrations.html

In [3]:
import os
import sys

In [4]:
path = '/content/drive/MyDrive/Colab Notebooks/Inteligencia Artificial/Grandes Modelos de Lenguaje'

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
sys.path.append(path)

In [11]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 9.4 MB/s eta 0:00:00


In [12]:
from config import api_key_hgface

In [13]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = api_key_hgface

In [14]:
from langchain import HuggingFaceHub

In [19]:
llm = HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":0, "max_length":64})
llm("Traducir de español a alemán: ¿Qué edad tienes?")

'Was ist Ihr Alter?'

In [2]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 5.3 MB/s eta 0:00:00


In [7]:
from config import api_key_openai

In [8]:
os.environ["OPENAI_API_KEY"] = api_key_openai

In [27]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9, model_name="text-davinci-003")
text = "¿Cuál sería un buen nombre para una startup disruptiva que ofrece servicios basados en inteligencia artificial?"
print(llm(text))



MindFlare AI.


## 2. Prompt Templates

LangChain facilita la gestión y la rápida optimización.

Normalmente, cuando usa un LLM en una aplicación, no está enviando la entrada del usuario directamente al LLM. En su lugar, debe tomar la entrada del usuario y construir un aviso, y solo luego enviarlo al LLM.

In [28]:
llm("¿Puede Andrés Manuel López Obrador tener una conversación con Porfirio Díaz?")

'\n\nNo, esto es imposible, ya que Porfirio Díaz murió en 1915 y Andrés Manuel López Obrador nació en 1953.'

In [29]:
prompt = """Question: ¿Puede Andrés Manuel López Obrador tener una conversación con Porfirio Díaz?

Pensemos paso a paso.

Respuesta: """
llm(prompt)

'\nNo, no es posible. Porfirio Díaz murió en 1915, mientras que Andrés Manuel López Obrador nació en 1953, lo que significa que hay una discrepancia de casi 40 años entre ambos. Además, en la actualidad, Porfirio Díaz no está vivo para tener una conversación con Andrés Manuel López Obrador.'

In [30]:
from langchain import PromptTemplate

template = """Pregunta: {pregunta}

Pensemos paso a paso.

Respuesta: """

prompt = PromptTemplate(template=template, input_variables=["pregunta"])

In [31]:
prompt.format(pregunta="¿Puede Andrés Manuel López Obrador tener una conversación con Porfirio Díaz?")

'Pregunta: ¿Puede Andrés Manuel López Obrador tener una conversación con Porfirio Díaz?\n\nPensemos paso a paso.\n\nRespuesta: '

In [32]:
try:
    llm(prompt)
except Exception as e:
    print("Requerimos de una cadena 'Chain'")

Requerimos de una cadena 'Chain'


## 3. Chains

Combina LLM y prompts en flujos de trabajo de varios pasos.

In [33]:
from langchain import LLMChain

In [34]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "¿Puede Andrés Manuel López Obrador tener una conversación con Porfirio Díaz?"
print(llm_chain.run(question))


No, Andrés Manuel López Obrador actualmente es el Presidente de México, mientras que Porfirio Díaz murió en 1915. Esto significa que no sería posible tener una conversación entre ellos.


## 4. Agentes y Herramientas

Los agentes involucran a un LLM que toma decisiones sobre qué acciones tomar, ejecutar esa acción, tener en cuenta una observación y repetirla hasta que se termine.

Cuando se usan correctamente, los agentes pueden ser extremadamente poderosos.Para cargar agentes se deben comprender los siguientes conceptos:

- Herramienta (tool): Función que realiza un deber específico. Esto puede ser por ejemplo: una búsqueda en Google, búsqueda en base de datos, Python REPL, entre otras cadenas.
- LLM: El modelo de lenguaje que impulsa al agente.
- Agente: El agente a utilizar.

In [35]:
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=2e2760e2bd7e329784f80413dc01d78d551a29cc3a15f2f1b936627a2b067696
  Stored in directory: /root/.cache/pip/wheels/c2/46/f4/caa1bee71096d7b0cdca2f2a2af45cacf35c5760bee8f00948
Successfully built wikipedia


In [36]:
from langchain.llms import OpenAI

from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [ ]:
llm = OpenAI(temperature=0)
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

In [ ]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [ ]:
agent.run("¿En qué año se estrenó la película Inception con Leonardo Dicaprio? ¿Cuál sería este año elevado a la potencia 0,43?")



> Entering new AgentExecutor chain...
 I need to find out the year the movie was released and then use the calculator to calculate the power.
Action: Wikipedia
Action Input: Inception (2010 film)
Observation: Page: Inception
Summary: Inception is a 2010 science fiction action film written and directed by Christopher Nolan, who also produced the film with Emma Thomas, his wife. The film stars Leonardo DiCaprio as a professional thief who steals information by infiltrating the subconscious of his targets. He is offered a chance to have his criminal history erased as payment for the implantation of another person's idea into a target's subconscious. The ensemble cast includes Ken Watanabe, Joseph Gordon-Levitt, Marion Cotillard, Elliot Page, Tom Hardy, Dileep Rao, Cillian Murphy, Tom Berenger, and Michael Caine.
After the 2002 completion of Insomnia, Nolan presented to Warner Bros. a written 80-page treatment for a horror film envisioning "dream stealers," based on lucid dreaming. Decid

'El año 2010 elevado a la potencia 0,43 es 26.3253591394933.'

## 5. Memoria

Agregar Estado a Cadenas y Agentes.

La memoria es el concepto de estado persistente entre llamadas de una cadena/agente. LangChain proporciona una interfaz estándar para la memoria, una colección de implementaciones de memoria y ejemplos de cadenas/agentes que usan memoria.

In [ ]:
from langchain import OpenAI, ConversationChain

In [ ]:
llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="Hola!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola!
AI:

> Finished chain.


' Hola! ¿Cómo estás? Me llamo AI. ¿Cómo te llamas?'

In [ ]:
conversation.predict(input="¿Podemos hablar de IA?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola!
AI:  Hola! ¿Cómo estás? Me llamo AI. ¿Cómo te llamas?
Human: ¿Podemos hablar de IA?
AI:

> Finished chain.


' ¡Claro! ¿Qué quieres saber sobre Inteligencia Artificial?'

In [ ]:
conversation.predict(input="Estoy interesado en la inteligencia artificial general")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola!
AI:  Hola! ¿Cómo estás? Me llamo AI. ¿Cómo te llamas?
Human: ¿Podemos hablar de IA?
AI:  ¡Claro! ¿Qué quieres saber sobre Inteligencia Artificial?
Human: Estoy interesado en la inteligencia artificial general
AI:

> Finished chain.


' La Inteligencia Artificial General se refiere a la creación de sistemas informáticos que pueden realizar tareas inteligentes sin necesidad de programación específica. Estos sistemas pueden aprender, planificar, razonar, entender el lenguaje y tomar decisiones.'

## 6. Cargadores de documentos

La combinación de modelos de lenguaje con sus propios datos de texto es una forma poderosa de diferenciarlos. El primer paso para hacer esto es cargar los datos en "documentos", una forma elegante de decir algunos fragmentos de texto. Este módulo tiene como objetivo hacer esto fácil.

https://python.langchain.com/en/latest/modules/indexes/document_loaders.html

In [44]:
!pip install unstructured
!pip install unstructured[local-inference]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 112.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [45]:
from langchain.document_loaders import OnlinePDFLoader

In [46]:
loader = OnlinePDFLoader("https://arxiv.org/pdf/1706.03762.pdf")
data = loader.load()

In [49]:
print (f'Existen {len(data)} documento (s) en el archivo')
print (f'Hay {len(data[0].page_content)} caracteres en el documento')

Existen 1 documento (s) en el archivo
Hay 39933 caracteres en el documento


## 7. Índices

Los índices se refieren a formas de estructurar documentos para que los LLM puedan interactuar mejor con ellos. Este módulo contiene funciones de utilidad para trabajar con documentos.

- Incrustaciones: Una incrustación es una representación numérica de una información, por ejemplo, texto, documentos, imágenes, audio, etc.
- Divisores de texto: Cuando desee manejar fragmentos largos de texto, es necesario dividir ese texto en fragmentos.
- Almacenes de vectores: Las bases de datos vectoriales almacenan e indexan incrustaciones de vectores de modelos NLP para comprender el significado y el contexto de cadenas de texto, oraciones y documentos completos para obtener resultados de búsqueda más precisos y relevantes.

In [ ]:
import requests

url = "https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)

In [ ]:
from langchain.document_loaders import TextLoader

In [ ]:
loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [ ]:
!pip install sentence_transformers

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings()

#text = "This is a test document."
#query_result = embeddings.embed_query(text)
#doc_result = embeddings.embed_documents([text])

In [ ]:
!pip install faiss-cpu

In [ ]:
# Almacen de vectores: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
from langchain.vectorstores import FAISS

In [ ]:
db = FAISS.from_documents(docs, embeddings)

query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

In [ ]:
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


In [ ]:
db.save_local("faiss_index")
new_db = FAISS.load_local("faiss_index", embeddings)
docs = new_db.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


**Por hacer**

* Probar el último ejemplo con alguna lectura en español

**Referencias**

* https://github.com/hwchase17/langchain
* https://huggingface.co/google/flan-t5-xl
* https://python.langchain.com/en/latest/index.html
* https://python.langchain.com/en/latest/modules/agents/tools.html
* https://python.langchain.com/en/latest/modules/agents/agents/agent_types.html
